In [265]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fuzzywuzzy # formator data
from fuzzywuzzy import process
from sklearn.impute import SimpleImputer # to change nan to mean value
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-sales-data/car_prices.csv


# Setup

In [266]:
# importing data
vehicle_location = '../input/vehicle-sales-data/car_prices.csv'
vehicle_data = pd.read_csv(vehicle_location)

# Showing table top 5 rows

In [267]:
vehicle_data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


# Droping useless columns

In [268]:
vehicle_data_formated = vehicle_data
vehicle_data_formated.drop(labels = ['interior','seller','trim','saledate','mmr','state'],axis = 1,inplace = True)

# Formating database

## Cleaning transmission

In [269]:
#function to find transmission in columns 
def find_transmission(row):
    for record in row:
        if record == 'automatic' or record == 'manual':
            return record
    return row['transmission']
#apply function and changing data
vehicle_data_formated['transmission'] = vehicle_data.apply(find_transmission, axis=1)
#checking what left in database
wrong_transmision = vehicle_data_formated[(
    vehicle_data_formated['transmission'] != 'automatic') & (
    vehicle_data_formated['transmission'] != 'manual') & (
    vehicle_data_formated['transmission'].isna() !=True)]
wrong_transmision

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
461612,2015,Volkswagen,Jetta,Navitgation,Sedan,NaN,NaN,46.0,2711,14250.0
505299,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,36.0,20379,13600.0
529009,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,2817,13750.0
551222,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,9562,13200.0


In [270]:
# i'll delete it cuz there is a lot of missing data
# creating index_transmision_to_delete
index_transmision_to_delete = wrong_transmision.index
# deleting rows
vehicle_data_formated.drop(index_transmision_to_delete, inplace=True)

In [271]:
# showing nan in transmision column
vehicle_data_formated[(vehicle_data_formated.transmission.isna())]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
35,2014,Buick,Verano,Sedan,NaN,1g4pr5sk8e4175320,2.0,19531.0,gray,9200.0
44,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sb6e7128803,NaN,38261.0,white,2000.0
89,2014,Chevrolet,Silverado 2500HD,Crew Cab,NaN,1gc1kvc84ef152736,36.0,8742.0,white,34000.0
112,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sbxe7113253,28.0,40393.0,gray,11700.0
281,2013,Hyundai,Sonata Hybrid,Sedan,NaN,kmhec4a4xda096513,48.0,1111.0,silver,15800.0
...,...,...,...,...,...,...,...,...,...,...
558814,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg3eh324197,43.0,20485.0,white,16100.0
558820,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg9eh324236,42.0,22744.0,white,15900.0
558829,2012,Hyundai,Elantra,Sedan,NaN,5npdh4ae7ch106397,4.0,66720.0,gray,10400.0
558830,2012,Nissan,Sentra,Sedan,NaN,3n1ab6ap3cl622485,26.0,35858.0,white,10400.0


In [272]:
# # taking all unique value 
# models_arr = vehicle_data_formated.model.unique()
# # main model
# def transmission_nan_changer(df:pd.DataFrame):
#     # lop for every string in array
#     for model_name in models_arr:
#         transmission_type = df[(df.model == model_name) & (df.transmission.isna() == False)].transmission.unique()
#         # indexes to change transmission
#         indexes = df[(df.model == model_name) & (df.transmission.isna() != False)].index
        
#         if len(transmission_type) != 0:
#             df.loc[indexes,'transmission'] = transmission_type[0]
#         else:
#             df.loc[indexes,'transmission'] = 'manual'
#     return df
# transmission_nan_changer(vehicle_data_formated)
# vehicle_data_formated.transmission.unique()


In [273]:
# # idexes of data that left 
# idexes_left_trans = vehicle_data_formated[(vehicle_data_formated.transmission.isna())].index
# vehicle_data_formated.drop(idexes_left_trans, inplace = True)

In [274]:
# showing nan in transmision column
vehicle_data_formated[(vehicle_data_formated.transmission.isna())]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
35,2014,Buick,Verano,Sedan,NaN,1g4pr5sk8e4175320,2.0,19531.0,gray,9200.0
44,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sb6e7128803,NaN,38261.0,white,2000.0
89,2014,Chevrolet,Silverado 2500HD,Crew Cab,NaN,1gc1kvc84ef152736,36.0,8742.0,white,34000.0
112,2014,Chevrolet,Cruze,Sedan,NaN,1g1pc5sbxe7113253,28.0,40393.0,gray,11700.0
281,2013,Hyundai,Sonata Hybrid,Sedan,NaN,kmhec4a4xda096513,48.0,1111.0,silver,15800.0
...,...,...,...,...,...,...,...,...,...,...
558814,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg3eh324197,43.0,20485.0,white,16100.0
558820,2014,Dodge,Charger,Sedan,NaN,2c3cdxbg9eh324236,42.0,22744.0,white,15900.0
558829,2012,Hyundai,Elantra,Sedan,NaN,5npdh4ae7ch106397,4.0,66720.0,gray,10400.0
558830,2012,Nissan,Sentra,Sedan,NaN,3n1ab6ap3cl622485,26.0,35858.0,white,10400.0


## Cleaning vin

In [275]:
def find_vin(row):
    for record in row:
        try:
            if len(record) == 17:
                return record
        except:
            pass
    return row['vin']
vehicle_data_formated['vin'] = vehicle_data.apply(find_vin, axis = 1)

In [276]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
494901,2010,Hyundai,Elantra,sedan,automatic,kmhdu4ad7au018421,36.0,60898.0,gray,8100.0
487556,2009,Chevrolet,Traverse,suv,NaN,1gner23d19s181351,35.0,87153.0,silver,11500.0
50992,2013,Ford,Escape,SUV,automatic,1fmcu9h94duc63577,2.0,35154.0,silver,16000.0
442995,2012,BMW,X5,SUV,automatic,5uxzw0c51cl672177,27.0,51221.0,black,29750.0
8251,2012,Toyota,Sienna,Minivan,NaN,5tddk3dc0cs031980,41.0,24897.0,gray,30400.0


In [277]:
# Checking what left
left_vin =  vehicle_data_formated[vehicle_data_formated.vin.str.len() != 17]
left_vin

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
408161,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,4802,13200.0
417835,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,1.0,9410,13300.0
421289,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,1167,13200.0
424161,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,1.0,2172,14050.0
427040,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,41.0,14872,13700.0
427043,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,49.0,12655,13850.0
434424,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,15719,13650.0
444501,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,2.0,6388,13850.0
453794,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,44.0,16633,13600.0
461597,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,44.0,11034,13900.0


In [278]:
# Deleting whats left 
vehicle_data_formated.drop(left_vin.index, inplace = True)

## Cleaning car makers

In [279]:
# deleting car without makers
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'].isnull() == False]

# taking all unique makers to variable
car_makers = vehicle_data_formated.make.unique()

# changing all value to lower case
np.unique(np.array([car.lower() for car in car_makers]))

array(['acura', 'airstream', 'aston martin', 'audi', 'bentley', 'bmw',
       'buick', 'cadillac', 'chev truck', 'chevrolet', 'chrysler',
       'daewoo', 'dodge', 'dodge tk', 'dot', 'ferrari', 'fiat', 'fisker',
       'ford', 'ford tk', 'ford truck', 'geo', 'gmc', 'gmc truck',
       'honda', 'hummer', 'hyundai', 'hyundai tk', 'infiniti', 'isuzu',
       'jaguar', 'jeep', 'kia', 'lamborghini', 'land rover', 'landrover',
       'lexus', 'lincoln', 'lotus', 'maserati', 'mazda', 'mazda tk',
       'mercedes', 'mercedes-b', 'mercedes-benz', 'mercury', 'mini',
       'mitsubishi', 'nissan', 'oldsmobile', 'plymouth', 'pontiac',
       'porsche', 'ram', 'rolls-royce', 'saab', 'saturn', 'scion',
       'smart', 'subaru', 'suzuki', 'tesla', 'toyota', 'volkswagen',
       'volvo', 'vw'], dtype='<U13')

In [280]:
# create function that will clear all car makers
def formating_car_makers(brand):
    #checking if brand is str
    if isinstance(brand, str):
        mapping = {
            'chev truck': 'chevrolet',
            'dodge tk': 'dodge',
            'gmc truck': 'gmc',
            'hyundai tk': 'hyundai',
            'land rover': 'landrover',
            'mazda tk': 'mazda',
            'mercedes-b': 'mercedes',
            'mercedes-benz': 'mercedes',
            'vw': 'volkswagen'
        }
        brand = brand.lower()
        if brand in mapping:
            return mapping[brand]
        else:
            return brand
    else:
        ''
# appklying function
vehicle_data_formated.loc[:,'make'] = vehicle_data.make.apply(lambda brand: formating_car_makers(brand))
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'] != 'dot']

In [281]:
# checking what left
vehicle_data_formated[vehicle_data_formated['make'].isna()]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice


## Cleaning car models

In [282]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
189973,2012,honda,Civic,Sedan,NaN,19xfb2f85ce384138,38.0,18019.0,burgundy,12000.0
470864,2007,saturn,Sky,convertible,automatic,1g8mb35b57y102018,25.0,101992.0,silver,3200.0
228677,2008,dodge,Caliber,Wagon,automatic,1b3hb28b18d532975,25.0,106035.0,black,4200.0
412111,2013,ford,Focus,Sedan,automatic,1fadp3f2xdl278549,27.0,56693.0,gray,8100.0
105871,2009,mercedes,GL-Class,SUV,automatic,4jgbf71e49a465863,44.0,68648.0,black,24500.0


In [283]:
# basic formating data 
vehicle_data_formated.loc[:,'model'] = vehicle_data_formated.model.str.lower()
vehicle_data_formated.loc[:,'model'] = vehicle_data_formated.model.str.strip()

In [284]:
# droping left nan value
model_to_drop = vehicle_data_formated[vehicle_data_formated['model'].isna()]
model_to_drop

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
72,2014,bmw,NaN,Sedan,automatic,wbaye8c59ed134514,43.0,12172.0,black,66000.0
31262,2014,bmw,NaN,Sedan,automatic,wbaya8c52ed825069,44.0,4488.0,silver,64200.0
61951,2014,bmw,NaN,Sedan,automatic,wbayf8c57ed653381,48.0,387.0,black,72500.0
61955,2014,bmw,NaN,Sedan,automatic,wbayf8c51ed142328,46.0,414.0,white,68500.0
61956,2014,bmw,NaN,Sedan,automatic,wbaya8c55ed228269,48.0,4796.0,NaN,75000.0
...,...,...,...,...,...,...,...,...,...,...
546744,2014,bmw,NaN,sedan,automatic,wbayf8c50ed142059,38.0,7394.0,black,56000.0
549010,2014,bmw,NaN,sedan,automatic,wbayf8c5xed142506,44.0,943.0,blue,64500.0
549011,2014,bmw,NaN,sedan,automatic,wbaye8c57ed780702,4.0,12895.0,white,57500.0
552124,2015,audi,NaN,sedan,automatic,waubfgffxf1005315,35.0,4529.0,gray,27000.0


In [285]:
vehicle_data_formated.drop(model_to_drop.index, inplace = True)

## Cleaning body

In [286]:
# basic formating data 
vehicle_data_formated.loc[:,'body'] = vehicle_data_formated.body.str.lower()
vehicle_data_formated.loc[:,'body'] = vehicle_data_formated.body.str.strip()

In [287]:
#creating copy of a vehicle_data_formated to have better vision of car make and model
db_to_count = vehicle_data_formated.copy()
db_to_count['make and model'] = db_to_count['make'] + ' ' + db_to_count['model']
db_to_count[db_to_count.body.isna()]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice,make and model
468,2013,lincoln,mkt,NaN,automatic,2lmhj5nk1dbl52290,41.0,74874.0,black,17750.0,lincoln mkt
743,2012,bmw,750i,NaN,automatic,wbakc6c5xcc395623,4.0,50790.0,gray,33500.0,bmw 750i
770,2012,bmw,750li,NaN,automatic,wbakb8c50cc963182,37.0,31762.0,black,45000.0,bmw 750li
793,2012,bmw,750i,NaN,automatic,wbaka8c54ccy37992,49.0,53016.0,white,34250.0,bmw 750i
794,2012,bmw,750li,NaN,automatic,wbakb8c56cc963431,34.0,24739.0,white,46750.0,bmw 750li
...,...,...,...,...,...,...,...,...,...,...,...
541919,2004,honda,accord,NaN,automatic,1hgcm66884a076293,35.0,199124.0,burgundy,3200.0,honda accord
548316,2010,chevrolet,malibu,NaN,automatic,1g1zb5eb7af274800,33.0,73483.0,gray,7400.0,chevrolet malibu
548323,2010,chevrolet,malibu,NaN,automatic,1g1zb5eb9af208345,33.0,59853.0,black,7200.0,chevrolet malibu
550571,2010,chevrolet,malibu,NaN,automatic,1g1zb5eb4af307705,28.0,76282.0,white,7400.0,chevrolet malibu


In [288]:
# counted car make and model 
db_to_count[db_to_count.body.isna()]['make and model'].value_counts()

make and model
chrysler pacifica        148
mitsubishi galant        100
lexus gx                  98
chevrolet hhr             92
honda pilot               82
                        ... 
chevrolet cavalier         1
ford 350                   1
landrover rrs              1
ford truck expedition      1
chevrolet uplandr          1
Name: count, Length: 220, dtype: int64

In [289]:
# function fixing car maker to reduce code
def changer_body(make, model, body):
    indexes = vehicle_data_formated[(vehicle_data_formated.make == make) & (vehicle_data_formated.model == model)].index
    vehicle_data_formated.loc[indexes,'body'] = body

# changing the most numerous records from nan
changer_body('chevrolet', 'malibu', 'sedan')
changer_body('bmw', '750i', 'sedan')
changer_body('chrysler', 'pacifica', 'minivan')
changer_body('lexus', 'gx', 'suv')
changer_body('chevrolet', 'hhr', 'suv')
changer_body('honda', 'pilot', 'suv')
changer_body('landrover', 'range', 'suv')
changer_body('cadillac', 'srx', 'suv')
changer_body('ford', 'explorer', 'suv')
changer_body('ford', 'f150', 'pickup')
changer_body('chevrolet', 'corvette', 'coupe')

In [290]:
# droping cars with body equal nan
indexes_to_drop_body = vehicle_data_formated[vehicle_data_formated.body.isna()].index
vehicle_data_formated.drop(indexes_to_drop_body,inplace = True)
vehicle_data_formated

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
0,2015,kia,sorento,suv,automatic,5xyktca69fg566472,5.0,16639.0,white,21500.0
1,2015,kia,sorento,suv,automatic,5xyktca69fg561319,5.0,9393.0,white,21500.0
2,2014,bmw,3 series,sedan,automatic,wba3c1c51ek116351,45.0,1331.0,gray,30000.0
3,2015,volvo,s60,sedan,automatic,yv1612tb4f1310987,41.0,14282.0,white,27750.0
4,2014,bmw,6 series gran coupe,sedan,automatic,wba6b2c57ed129731,43.0,2641.0,gray,67000.0
...,...,...,...,...,...,...,...,...,...,...
558832,2015,kia,k900,sedan,NaN,knalw4d4xf6019304,45.0,18255.0,silver,33000.0
558833,2012,ram,2500,crew cab,automatic,3c6td5et6cg112407,5.0,54393.0,white,30800.0
558834,2012,bmw,x5,suv,automatic,5uxzw0c58cl668465,48.0,50561.0,black,34000.0
558835,2015,nissan,altima,sedan,automatic,1n4al3ap0fc216050,38.0,16658.0,white,11100.0


## Condition cleaning

In [291]:
# checking nan value
condition_nan = vehicle_data_formated[vehicle_data_formated.condition.isna()]
condition_nan

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
14,2014,chevrolet,cruze,sedan,automatic,1g1pe5sbxe7120097,NaN,15686.0,blue,10600.0
16,2015,hyundai,sonata,sedan,automatic,5npe24af4fh001562,NaN,8311.0,red,4200.0
22,2014,chevrolet,camaro,convertible,automatic,2g1fb3d31e9134662,NaN,33450.0,black,14700.0
25,2015,hyundai,sonata,sedan,automatic,5npe24af4fh038482,NaN,9281.0,silver,8500.0
28,2014,bmw,x5,suv,automatic,5uxkr2c52e0h33130,NaN,11278.0,gray,34000.0
...,...,...,...,...,...,...,...,...,...,...
43570,2010,acura,rdx,suv,automatic,5j8tb1h59aa000847,NaN,63454.0,black,14500.0
43579,2011,toyota,tundra,double cab,automatic,5tfrm5f17bx031104,NaN,46683.0,black,19250.0
43580,2011,volkswagen,jetta,sedan,automatic,3vwdz7aj1bm338415,NaN,30778.0,black,9500.0
182059,2005,ford,f-150,supercab,NaN,1ftpx14575na62365,NaN,264448.0,white,1000.0


In [292]:
# best option is to drop these records
vehicle_data_formated.drop(condition_nan.index, inplace = True)

## Odometer cleaning

In [293]:
# checking nan value
condition_nan = vehicle_data_formated[vehicle_data_formated.odometer.isna()]
condition_nan

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
3360,2007,ford,edge,suv,NaN,2fmdk38c77bb39171,32.0,NaN,gray,15500.0
5119,2000,mercedes,m-class,suv,automatic,4jgab54e5ya217238,26.0,NaN,white,1500.0
7566,2002,ford,f-350 super duty,crew cab,NaN,1ftsw31f52ed16477,2.0,NaN,white,1.0
22558,2005,ford,focus,sedan,automatic,1fafp34n95w279634,19.0,NaN,black,1600.0
27791,2001,buick,regal,sedan,automatic,2g4wb55k211127121,19.0,NaN,silver,500.0
39694,2007,chevrolet,colorado,crew cab,automatic,1gccs13e078206645,19.0,NaN,gold,7800.0
47640,2005,mercury,montego,sedan,automatic,1mefm40195g624858,19.0,NaN,red,650.0
48973,2001,mercedes,c-class,sedan,manual,wdbrf61j71f025555,19.0,NaN,burgundy,1200.0
49015,2000,buick,lesabre,sedan,NaN,1g4hp54k7y4237611,1.0,NaN,white,1250.0
129505,2005,ford,freestyle,wagon,automatic,1fmdk06125ga70556,19.0,NaN,black,2100.0


In [294]:
# few records so best option is to drop them
vehicle_data_formated.drop(condition_nan.index, inplace = True)

## Color cleaning

In [295]:
# checking nan value
color_nan = vehicle_data_formated[vehicle_data_formated.color.isna()]
color_nan

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
1824,2011,bmw,m3,sedan,automatic,wbspm9c59be699111,4.0,26406.0,NaN,42500.0
2257,2011,mercedes,c-class,sedan,automatic,wddgf5gbxbr146489,36.0,34961.0,NaN,21500.0
2334,2011,mercedes,cls-class,sedan,automatic,wdddj7cbxba169305,35.0,27372.0,NaN,30000.0
2389,2011,mercedes,s-class,sedan,automatic,wddng7eb3ba409875,35.0,8120.0,NaN,66000.0
6043,2013,bmw,3 series,sedan,automatic,wba3a5c53df600381,49.0,49997.0,NaN,24250.0
...,...,...,...,...,...,...,...,...,...,...
548377,2015,mini,cooper countryman,wagon,automatic,Cooper Countryman,47.0,3569.0,NaN,31250.0
550197,2012,nissan,frontier,crew cab,automatic,1n6ad0er2cc456064,34.0,87082.0,NaN,15500.0
553040,2011,ford,f-150,supercrew,automatic,1ftfw1et2bfa51808,19.0,67302.0,NaN,27500.0
553525,2014,chevrolet,equinox,suv,automatic,1gnaldek6ez103530,48.0,24425.0,NaN,23900.0


In [296]:
# also as before only 632 records so i`ll drop them
vehicle_data_formated.drop(color_nan.index, inplace = True)

## Selling price cleaning